Colab para generar clase_ternaria y obtener el archivo competencia_02.csv.gz para subir al bucket de google cloud (extraido de la ayudita que dio el profe Alejandro en Zullip)

## IMPORTANTE: SUBIR EL ARCHIVO competencia_02_crudo.csv.gz al entorno del Google Colab

In [ ]:
%%bash
pip install --upgrade duckdb
pip install --upgrade jupysql
pip install --upgrade duckdb-engine

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 44.2 MB/s eta 0:00:00
  Attempting uninstall: duckdb
    Found existing installation: duckdb 0.8.1
    Uninstalling duckdb-0.8.1:
      Successfully uninstalled duckdb-0.8.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.9/315.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.5/376.5 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 kB 1.4 MB/s eta 0:00:00


"A diferencia de la competencia 1, ahora vamos a usar una base de datos en disco. Todo va a ser más lento, pero se va a poder trabajar sin problemas en máquinas de menor capacidad. La DB va a queda en el archivo dmeyf.db, lo probé en un colab básico y funcionó sin problemas."


In [ ]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:////content/dmeyf.db

"Cargamos el dataset, y como tenemos la base en el disco, no debemos cargarlo más, la próxima se puede usar directamente"


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%sql
create or replace table competencia_02 as
select
    *
from read_csv_auto('/content/drive/MyDrive/DM/DMEyF/competencia_02_crudo.csv.gz')


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


"El ayudin. Que hace el siguiente código:

Toma todos los valores de foto_mes y numero_de_cliente
Cruza todos con todos. De esta manera nos aseguramos que va haber un registro para todos los clientes en todos los periodos.
Luego se crean las variables soporte mes_0, que nos dice si el cliente esta en esa foto_mes, mes_1 si esta en la foto_mes siguiente y mes_2 si esta en las siguientes dos foto_mes
Debe completar su lógica en la variable clase_ternaria usando estas variables auxiliares
La tabla final se queda solamente con las variables y los registros necesarios."


#### Lógica que utilicé: Si mes_0 = 1, mes_1 = 0, mes_2 = 0  entonces BAJA +1 / Si mes_0=1, mes_1=1, mes_2 = 0, entonces BAJA+2 / Si mes_0=1, mes_1 = 0 , mes_2= 1 entonces CONTINUA / los demas casos son CONTINUA

In [ ]:
%%sql
create or replace table targets as
with periodos as (
    select distinct foto_mes from competencia_02
), clientes as (
    select distinct numero_de_cliente from competencia_02
), todo as (
    select numero_de_cliente, foto_mes from clientes cross join periodos
), clase_ternaria as (
    select
        t.numero_de_cliente
        , t.foto_mes
        , if(c.numero_de_cliente is null, 0, 1) as mes_0
        , lead(mes_0, 1) over (partition by t.numero_de_cliente order by foto_mes) as mes_1
        , lead(mes_0, 2) over (partition by t.numero_de_cliente order by foto_mes) as mes_2
        , CASE
            WHEN mes_0 = 1 AND mes_1 = 0 AND mes_2 = 0 THEN 'BAJA+1'
            WHEN mes_0 = 1 AND mes_1 = 1 AND mes_2 = 0 THEN 'BAJA+2'
            WHEN mes_0 = 1 AND mes_1 = 0 AND mes_2 = 1 THEN 'CONTINUA'
            ELSE 'CONTINUA'
          END as clase_ternaria
    from todo t
    left join competencia_02 c using (numero_de_cliente, foto_mes)
) select
  foto_mes
  , numero_de_cliente
  , clase_ternaria
from clase_ternaria where mes_0 = 1


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


"Luego agregamos la variable clase_ternaria a nuestra tabla y cargamos los datos que previamente calculamos."


In [ ]:
%%sql
alter table competencia_02 add column clase_ternaria VARCHAR(10)


,Success


In [ ]:
%%sql
update competencia_02
set clase_ternaria = targets.clase_ternaria
from targets
where competencia_02.numero_de_cliente = targets.numero_de_cliente and competencia_02.foto_mes = targets.foto_mes


,Success


"Y finalmente guardamos el dataset para usarlo en la competencia. Esto les va a tardar en guardar."


In [ ]:
%%sql
copy competencia_02 to '/content/competencia_02.csv.gz' (FORMAT CSV, HEADER)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


#### Importante: Hay que descargarse el archivo "competencia_02.csv.gz" generado y subirlo al bucket de google cloud

##### Para chequear que esté bien creada clase_ternaria

In [ ]:
%%sql
create or replace table competencia_02 as
select
    *
from read_csv_auto('/content/competencia_02.csv.gz')

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [ ]:
%%sql
SELECT * FROM competencia_02 WHERE clase_ternaria IS NOT NULL LIMIT 200

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria
0,29271095,201901,1,0,0,77,22,-7456.29,-1552.35,-5180.00,...,1.0,0.0,NaN,NaN,638.0,NaN,NaN,NaN,0.00,BAJA+2
1,29273170,201901,1,0,1,71,263,1620.60,17877.03,1377.58,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BAJA+2
2,29743180,201901,1,0,1,78,349,4476.80,26803.42,1144.06,...,1.0,0.0,-68972.40,0.0,8502.0,59667.42,48.0,0.0,14979.21,BAJA+2
3,29913198,201901,1,0,0,58,253,19516.87,140306.82,299.43,...,8.0,0.0,NaN,NaN,6349.0,NaN,NaN,NaN,0.00,BAJA+2
4,29988202,201901,1,0,1,62,16,3265.54,43567.06,5546.77,...,1.0,0.0,0.00,0.0,217.0,0.00,0.0,0.0,0.00,BAJA+2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,69188257,201901,1,0,1,46,175,348.15,15316.42,185.75,...,1.0,0.0,0.00,0.0,3222.0,3775.80,3.0,0.0,408579.36,BAJA+2
196,69198351,201901,1,0,1,55,163,33.53,4367.73,33.27,...,1.0,0.0,NaN,NaN,4435.0,NaN,NaN,NaN,0.00,BAJA+2
197,69246267,201901,1,0,1,65,175,928.60,4337.51,325.97,...,8.0,0.0,-7214.90,0.0,4733.0,5983.48,3.0,0.0,445.74,BAJA+2
198,69333065,201901,1,0,1,40,151,4090.15,47557.18,1563.23,...,1.0,0.0,-65121.41,0.0,3479.0,9065.72,7.0,0.0,0.00,BAJA+2
